In [3]:
class World:
    def __new__(cls):
        if hasattr(cls, 'instance'):
            return cls.instance
        else:
            cls.instance = super().__new__(cls)
            return cls.instance


w1 = World()
w2 = World()
print(id(w1), id(w2))

1962685286784 1962685286784


#### Decorators in Python

A syntax feature that aids in AOP patterns in a simpler syntax / structure 

In [12]:
def to_upper(fn):
    def wrap():
        return fn().upper()
    return wrap

def strong(fn):
    def wrap():
        return "<strong>" + fn() + "</strong>"
    return wrap




In [16]:
@strong
@to_upper
def greet():
    return "Hello, world"

#greet = to_upper(greet)
greet()

'<strong>HELLO, WORLD</strong>'

In [10]:
def decorator(fn):
    print("decorator invoked")
    def wrap():
        print("wrap invoked")
        fn()
    return wrap

@decorator
def testfn():
    print("testfn invoked")


testfn()
testfn()
testfn()

decorator invoked
wrap invoked
testfn invoked
wrap invoked
testfn invoked
wrap invoked
testfn invoked


In [ ]:
@profile
def slow_test(count):
    for i in range(count):
        pass

slow_test(100_000_000)
slow_test(10_000_000)
slow_test.report()

# OUT: Wed Aug 13 10:43:15 2025: slow_test(100_000_000) took 1.23 seconds
# OUT  Wed Aug 13 10:43:16 2025: slow_test(10_000_000) took 0.27 seconds


In [20]:
from time import ctime
ctime()

'Wed Aug 13 10:43:15 2025'